# 내컴퓨터의 동영상 썸네일 만들기

* 난이도 : ★★★★☆☆☆☆☆☆
* 필요라이브러리: os, mimetypes, argparse, cv2, numpy


* 하나의 동영상 파일에서 여러장의 이미지를 추출하여 한장의 썸네일을 만듭니다.(콜라주 같은...)
* 파일이 동영상인지를 정확하게 확인하는건 간단한 문제는 아닙니다. 여기서는 일반적인 파일의 확장자로 파일의 MIMETYPE을 알아오는 파이썬 기본 라이브러리인 mimetypes 라는 라이브러리를 사용하도록 하겠습니다.
* mimetyps 공식 문서 링크 : https://docs.python.org/3.6/library/mimetypes.html

### openCV 로 동영상 파일 읽어보기
* 파일의 MIMETYPE 을 알아낸 후 동영상 파일일때 수행합니다.
* openCV 의 VideoCapture 클래스를 통해 동영상의 여러가지 정보를 추출해봅니다.
* openCV 의 VideoCapture 문서 참조: https://docs.opencv.org/java/3.0.0/org/opencv/videoio/VideoCapture.html
* <b style='color:red'>opencv 라이브러리를 통해 동영상을 open 하려면 반드시 해당 컴퓨터에 코덱이 설치 되어있어야 합니다.</b>

In [ ]:
'''
openCV 로 동영상 파일의 정보를 알아오는 예제
'''
import cv2
import mimetypes

# 동영상 파일 경로
filepath = "movie.mp4"

# mimetypes 라이브러리의 guess_type 함수를 사용해
# 해당 파일의 mimetype 을 알아봅니다.
# 결과는 ('video/mp4', None) 식의 튜플 형태로 리턴되니 0번째 타입만 받아서 사용합니다.
mimetype = mimetypes.guess_type(filepath)[0]
print(mimetype)

# mimetype 이 video 인 경우만
if mimetype.find("video") >= 0:
    # openCV 의 VideoCapture 클래스를 생성합니다.
    cap = cv2.VideoCapture(filepath)

    # 동영상의 전체 프레임수
    v_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 동영상의 사이즈 정보
    v_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    v_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # 동영상의 FPS
    v_fps = cap.get(cv2.CAP_PROP_FPS)

    # 동영상의 총 재생시간
    # 총 재생시간은 전체 프레임 수 / 초당 프레임수
    v_duration = v_frames / v_fps
    # v_duration 을 분/초로 계산
    v_min = int(v_duration / 60)
    v_sec = int(v_duration % 60)

    print("=" * 70)
    print("대상파일: {}".format(filepath))
    print("영상사이즈: {} x {}".format(v_width, v_height))
    print("총 재생시간: {} 분 {} 초 ({}sec)".format(v_min, v_sec, v_duration))
    print("전체 프레임 수: {}".format(v_frames))
    print("초당 프레임 수: {}".format(v_fps))

    # read() 함수를 호출하면 호출결과, 프레임을 리턴합니다.
    # 여기서 read() 함수는 프레임이 위치한 곳의 프레임을 리턴합니다.(최초 0프레임부터)
    # read() 함수를 호출하고 나면 cap 의 포인터는 다음 프레임을 가르키게 됩니다.(1 프레임으로 이동)
    ret, frame = cap.read()
    if ret:
        cv2.imshow("frame", frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

### 동영상에서 여러장의 이미지를 뽑아내기
* 원하는 이미지 수를 인자로 받아 해당 이미지 수 만큼 동영상에서 이미지를 추출하는 기능을 구현합니다.
* 추출된 이미지는 opencv 이미지 리스트로 리턴되게 합니다.

In [ ]:
import cv2

def capture_video(filepath, capture_count=1):
    '''filepath 로 넘어온 파일 경로의 동영상 정보를 리턴하는 함수 입니다.
    
    Args:
        filepath (str) : 동영상 파일 경로
        capture_count (int) : 캡쳐 이미지 수
        
    Returns:
        list : opencv 이미지 리스트 (파일 목록이 아님!!)
    '''
    
    # 최종 리턴될 이미지 목록 리스트 변수
    images = []
    
    try:
        # openCV 의 VideoCapture 클래스를 생성합니다.
        cap = cv2.VideoCapture(filepath)

        # 동영상의 전체 프레임수
        v_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 동영상의 FPS
        v_fps = cap.get(cv2.CAP_PROP_FPS)
        
        # 캡처를 하기 위해 얼만큼 프레임을 건너띄어야 하는지 계산합니다.
        # 전체 프레임 수 / 캡쳐 장수 = 건너띌 프레임 수
        # 100 프레임을 5장으로 나누면
        # 1, 21, 41, 61, 81 이런 결과를 얻기 위함 입니다.
        jump_frame = int(v_frames / capture_count)
        
        # 캡쳐 장수만큼 반복
        for i in range(capture_count):
            # 최초 프레임은 1부터 
            pos = 1 + (jump_frame * i)
            
            # 프레임의 위치를 캡쳐할 위치로 설정합니다.
            cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
            
            # 해당 프레임을 read() 합니다.
            ret, frame = cap.read()

            # ret 가 True 면
            if ret:
                # 해당 프레임 데이터를 images 리스트에 추가 합니다.
                images.append(frame)
                
        # open 된 동영상 객체를 release 시킵니다.
        cap.release()
    except:
        pass
    
    # 최종 이미지 리스트를 리턴합니다.
    return images

### 콜라주 만들기
* 여러장의 이미지를 인자로 받아 해당 이미지를 한장의 콜라주로 만드는 기능을 구현합니다.

In [ ]:
# 이미지 처리를 위해 numpy 라이브러리를 임포트 한 후 이름을 np 라고 짓습니다.
# 이는 코딩이 좀 더 간결해짐을 목적일 뿐 입니다.
import numpy as np

def create_collage(image_list, thumb_size=(210, 137), rowcols=(5, 5)):
    '''이미지 리스트를 받아서 한장의 콜라주를 만들어주는 함수
    
    Args:
        image_list (list) : opencv_image 목록 입니다.(파일목록이 아닙니다.)
        thumb_size (tuple) : 콜라주에 들어갈 가로 x 세로의 썸네일 사이즈 입니다.
        rowcols (tuple) : 콜라주의 행, 렬을 설정합니다. 기본값은 5 x 5 짜리 콜라주 입니다.
    '''
    
    # 새로 생성될 이미지는 썸네일 가로 * cos 의 갯수 의 width 값을 갖고
    # 썸네일 세로크기 * row 갯수 의 height 값을 갖습니다.
    canvas_width = thumb_size[0] * rowcols[0]
    canvas_height = thumb_size[1] * rowcols[1]
    
    # 여기서 중요한 부분이 이미지로쓸 numpy 배열을 생성하는데
    # shape (배열의 형태) 를 보면 이미지의 height, width, 채널수 입니다. 
    # 보통 이미지 처리에선 width, height 으로 이미지 크기를 표기하는데
    # numpy, openCV 에서는 height 값이 먼저 입니다.
    # 채널 3은 컬러 이미지인 경우 R, G, B 의 3개 채널을 갖게 됩니다.
    # 결론적으로 canvas는 3차원 배열 형태입니다.
    canvas = np.zeros(shape=(canvas_height, canvas_width, 3), dtype=np.uint8)
    
    # 콜라주 이미지의 배경색상을 흰색으로 설정합니다.
    canvas.fill(255)
    
    # 새로생성된 콜라주에 썸네일 이미지를 붙여넣기 할때
    # 현재 어느 위치에 붙여넣기를 해야할지 기억할 변수를 설정합니다.
    cursor = [0, 0]
    
    # 이미지 목록만큼 반복합니다.
    for img in image_list:
        # 썸네일 사이즈로 이미지를 리사이즈 합니다.
        img = cv2.resize(img, thumb_size)
        
        # 3차원 배열 형태의 openCV 이미지에서 특정 영역을 접근하는 방식은 
        # 쉽게 image[y:y+1 , x:x+1] 로 생각하면 됩니다.
        # 커서[높이] : 커서[높이] + 썸네일 높이
        # 커서[폭] : 커서[폭] + 썸네일 폭
        # 의 배열값에 img 배열값을 붙여넣기 한다는 이야기 입닏.
        canvas[cursor[1]:cursor[1] + thumb_size[1], cursor[0]:cursor[0] + thumb_size[0]] = img
        
        # canvas에 현재 썸네일 이미지를 붙여넣기 후에는 
        # 다음이미지를 위해 커서값을 썸네일 width 값 만큼 이동시켜 기억하고 있어야 합니다.
        # 예를 들어 100픽셀 이미지를 붙여넣기 했다면 처음엔 0의 위치에서 100만큼을 이동했으니
        # 다음 이미지를 붙여넣기 할때는 100번 위치 부터 해야하기 때문입니다.
        cursor[0] += thumb_size[0]
        
        # 커서의 폭 값이 행 * 썸네일폭 보다 크면 다음줄로 내려가서 붙여넣기 해야 합니다.
        if cursor[0] >= rowcols[0] * thumb_size[0]:
            # 다음 열의 시작위치는 썸네일 높이 만큼 증가해야 합니다.
            cursor[1] += thumb_size[1] 
            
            # 다음 열로 내려간 후에는 좌측 처음부터 다시 시작해야 하기 때문에
            # 커서[폭]의 값을 0 으로 초기화 합니다.
            cursor[0] = 0
    
    # 만들어진 canvas를 리턴합니다.
    return canvas

# 시나리오

1. 대상 폴더 하위의 모든 폴더를 검색하여 동영상 파일을 찾아 리스트로 리턴합니다.
2. 동영상 파일 목록을 반복하며 해당 동영상에서 정해진 수 만큼의 이미지를 뽑아 냅니다.
3. 뽑아낸 이미지를 갖고 한장의 콜라주 이미지를 만듭니다.
4. 콜라주 이미지를 원본 동영상 파일명.jpg 로 저장합니다.

### 최종 완성 코드
* 대상폴더, 대상 단일 파일 옵션을 추가

In [5]:
import cv2
import os
import mimetypes
import numpy as np
import argparse


def capture_video(filepath, capture_count=1):
    '''filepath 로 넘어온 파일 경로의 동영상 정보를 리턴하는 함수 입니다.
    Args:
        filepath (str) : 동영상 파일 경로
        capture_count (int) : 캡쳐 이미지 수
    Returns:
        list : opencv 이미지 리스트 (파일 목록이 아님!!)
    '''
    
    # 최종 리턴될 이미지 목록 리스트 변수
    images = []
    
    try:
        # openCV 의 VideoCapture 클래스를 생성합니다.
        cap = cv2.VideoCapture(filepath)

        # 동영상의 전체 프레임수
        v_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 동영상의 FPS
        v_fps = cap.get(cv2.CAP_PROP_FPS)
        
        # 캡처를 하기 위해 얼만큼 프레임을 건너띄어야 하는지 계산합니다.
        # 전체 프레임 수 / 캡쳐 장수 = 건너띌 프레임 수
        # 100 프레임을 5장으로 나누면
        # 1, 21, 41, 61, 81 이런 결과를 얻기 위함 입니다.
        jump_frame = int(v_frames / capture_count)
        
        # 캡쳐 장수만큼 반복
        for i in range(capture_count):
            # 최초 프레임은 1부터 
            pos = 1 + (jump_frame * i)
            
            # 프레임의 위치를 캡쳐할 위치로 설정합니다.
            cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
            
            # 해당 프레임을 read() 합니다.
            ret, frame = cap.read()

            # ret 가 True 면
            if ret:
                # 해당 프레임 데이터를 images 리스트에 추가 합니다.
                images.append(frame)
                
        # open 된 동영상 객체를 release 시킵니다.
        cap.release()
    except:
        pass
    
    # 최종 이미지 리스트를 리턴합니다.
    return images
    
    
def search_dir(dirname, only_movie=True):
    '''대상 폴더 내의 파일과 서브 폴더를 탐색하는 재귀 함수

    Args:
        dirname (str): 탐색 대상 폴더 경로
        only_move (bool) : True명 동영상 파일만 대상으로 합니다.

    Returns:
        list : 탐색한 결과 파일 리스트를 리턴
    '''
    # 결과를 리턴할 리스트 변수
    result_file_lists = []

    # dirname 의 경로의 파일과 폴더 목록을 구함
    filenames = os.listdir(dirname)

    # 파일과 폴더 목록 반복문
    for filename in filenames:
        # filename 에는 os.listdir(dirname) 에서 dirname 이후의 경로값과 파일명만 존재하기 때문에
        # 전체 풀 경로를 얻기 위해선 dirname 과 filename 을 합쳐야 함
        full_filepath = os.path.join(dirname, filename)

        # full_filepath 가 디렉토리라면..
        if os.path.isdir(full_filepath):
            # search_dir 인 자기 자신 함수를 다시 호출한다.(재귀함수)
            result_file_lists.extend(search_dir(full_filepath, only_movie=only_movie))
        # 파일인 경우 result_file_lists 에 추가
        else:
            # mimetyps 라이브러리를 사용하여 mimetype 을 구합니다.
            # mimetypes[0] 이 None 인경우는 mimetype = "" 빈문자열 값이 들어가고
            # 그렇지 않으면 mimetype = mimetypes[0] 이 들어 갑니다.
            # 이전 강좌에서 다룬 if else 를 한줄로 작성하는 스타일 입니다.
            mimetype = "" if mimetypes.guess_type(full_filepath)[0] is None else mimetypes.guess_type(full_filepath)[0]
            
            # only_movie 가 True 고 mimetype 에 video 글자가 존재하는 경우
            # 혹은 only_movie 가 False 인 경우
            if (only_movie and mimetype.find("video") >= 0) or not only_movie:
                result_file_lists.append(full_filepath)

    # 결과 목록 리턴
    return result_file_lists

    
def create_collage(image_list, thumb_size=(210, 137), rowcols=(4, 5)):
    '''이미지 리스트를 받아서 한장의 콜라주를 만들어주는 함수
    
    Args:
        image_list (list) : opencv_image 목록 입니다.(파일목록이 아닙니다.)
        thumb_size (tuple) : 콜라주에 들어갈 가로 x 세로의 썸네일 사이즈 입니다.
        rowcols (tuple) : 콜라주의 행, 렬을 설정합니다. 기본값은 4 x 5 짜리 콜라주 입니다.
    '''
    
    # 새로 생성될 이미지는 썸네일 가로 * cos 의 갯수 의 width 값을 갖고
    # 썸네일 세로크기 * row 갯수 의 height 값을 갖습니다.
    canvas_width = thumb_size[0] * rowcols[0]
    canvas_height = thumb_size[1] * rowcols[1]
    
    # 여기서 중요한 부분이 이미지로쓸 numpy 배열을 생성하는데
    # shape (배열의 형태) 를 보면 이미지의 height, width, 채널수 입니다. 
    # 보통 이미지 처리에선 width, height 으로 이미지 크기를 표기하는데
    # numpy, openCV 에서는 height 값이 먼저 입니다.
    # 채널 3은 컬러 이미지인 경우 R, G, B 의 3개 채널을 갖게 됩니다.
    # 결론적으로 canvas는 3차원 배열 형태입니다.
    canvas = np.zeros(shape=(canvas_height, canvas_width, 3), dtype=np.uint8)
    
    # 콜라주 이미지의 배경색상을 흰색으로 설정합니다.
    canvas.fill(255)
    
    # 새로생성된 콜라주에 썸네일 이미지를 붙여넣기 할때
    # 현재 어느 위치에 붙여넣기를 해야할지 기억할 변수를 설정합니다.
    cursor = [0, 0]
    
    # 이미지 목록만큼 반복합니다.
    for img in image_list:
        # 썸네일 사이즈로 이미지를 리사이즈 합니다.
        img = cv2.resize(img, thumb_size)
        
        # 3차원 배열 형태의 openCV 이미지에서 특정 영역을 접근하는 방식은 
        # 쉽게 image[y:y+1 , x:x+1] 로 생각하면 됩니다.
        # 커서[높이] : 커서[높이] + 썸네일 높이
        # 커서[폭] : 커서[폭] + 썸네일 폭
        # 의 배열값에 img 배열값을 붙여넣기 한다는 이야기 입닏.
        canvas[cursor[1]:cursor[1] + thumb_size[1], cursor[0]:cursor[0] + thumb_size[0]] = img
        
        # canvas에 현재 썸네일 이미지를 붙여넣기 후에는 
        # 다음이미지를 위해 커서값을 썸네일 width 값 만큼 이동시켜 기억하고 있어야 합니다.
        # 예를 들어 100픽셀 이미지를 붙여넣기 했다면 처음엔 0의 위치에서 100만큼을 이동했으니
        # 다음 이미지를 붙여넣기 할때는 100번 위치 부터 해야하기 때문입니다.
        cursor[0] += thumb_size[0]
        
        # 커서의 폭 값이 행 * 썸네일폭 보다 크면 다음줄로 내려가서 붙여넣기 해야 합니다.
        if cursor[0] >= rowcols[0] * thumb_size[0]:
            # 다음 열의 시작위치는 썸네일 높이 만큼 증가해야 합니다.
            cursor[1] += thumb_size[1] 
            
            # 다음 열로 내려간 후에는 좌측 처음부터 다시 시작해야 하기 때문에
            # 커서[폭]의 값을 0 으로 초기화 합니다.
            cursor[0] = 0
    
    # 만들어진 canvas를 리턴합니다.
    return canvas

def my_imwrite(filename, img):
    '''opencv 의 imwrite() 함수에서 파일명이 한글인경우 오류가 발생합니다.
    이는 opencv 자체 버그라 직접 데이터를 인코딩 해서 저장하는 함수로 대체 하는 함수 입니다.

    Args:
        filename (str) : 파일명
        img (numpy.ndarry) : openCV 이미지

    Returns:
        bool : True or False
    '''
    try:
        # 파일명에서 확장자를 분리합니다.
        # 이는 openCV 의 imencode 함수의 인자로 들어가서 
        # ext 형태에 맞게 인코딩을 해주는 역할을 합니다. (jpg, gif...)
        ext = os.path.splitext(filename)[1]

        # imencode 함수는 (bool, numpy.ndarray) 를 리턴합니다.
        result, buffer = cv2.imencode(ext, img)

        # imencode() 호출시 result 가 True 면
        if result:
            # 인자로 넘어온 파일명을 wb 형태로 오픈합니다.(Write + Binary)
            with open(filename, mode="w+b") as f:
                # ndarray 를 파일로 기록합니다.
                buffer.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False
    
    
parse = argparse.ArgumentParser()
parse.add_argument("-folder", type=str, help="대상폴더")
parse.add_argument("-file", type=str, help="대상파일")
parse.add_argument("-c", type=int, default=20, help="동영상에서 추출할 이미지 장 수")
parse.add_argument("-size", nargs='+', type=int, help="썸네일 사이즈 210 137")
parse.add_argument("-rowcols", nargs='+', type=int, help="콜라주 행과 열 5 4")
args = parse.parse_args()


if args.folder is None and args.file is None:
    print("사용법: python video_thumbnail.py -folder <대상폴더> or -file <대상파일>")
else:
    # 이 문장을 한줄로 표현
    # if args.size is None:
    #     size = (210, 137)
    # else:
    #     size = tuple(args.size)
    size = (210, 137) if args.size is None else tuple(args.size)
    rowcols = (5, 4) if args.rowcols is None else tuple(args.rowcols)

    if args.folder is not None:
        # 실행인자의 f 로 넘어온 대상 폴더에서 동영상 파일목록 추출
        file_list = search_dir(args.f, only_movie=True)
    else:
        file_list = [args.file]
    
    # 파일목록 요소 반복
    for file in file_list:
        # 해당 파일에서 args.c 카운트 만큼 프레임을 추출합니다.
        images = capture_video(file, capture_count=args.c)
        
        # 추출된 프레임수가 0보다 크면
        if len(images) > 0:
            # 동영상 파일명에서 확장자만 바꿔서 이미지 파일명 생성
            new_filename = os.path.splitext(file)[0] + ".jpg"
            
            # 해당 이미지 목록을 한장의 콜라주 이미지로 생성합니다.
            collage = create_collage(images, thumb_size=size, rowcols=rowcols)
            
            # 생성된 콜라주 이미지 파일로 저장 
            # 원래는 opencv 의 imwrite() 함수로 저장합니다만
            # 파일경로에 유니코드(한글)가 들어가면 저장이 되지 않는 오류가 있습니다.
            # 그래서 직접 이미지를 저장하는 함수를 만들어 호출합니다.
            # cv2.imwrite(new_filename, collage)
            my_imwrite(new_filename, collage)
            print("{} 저장 완료..".format(new_filename))


c:\temp\movie.jpg 저장 완료..
